In [1]:
import os

In [2]:
import librosa

In [3]:
import mir_eval

In [4]:
from collections import OrderedDict

In [5]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=3)
pd.set_option('precision', 4, "display.max_rows", 999)

In [6]:
def make_onset_corpus(onset_path):
    
    # Beat files
    audio = librosa.util.find_files(onset_path, ext='wav')
    
    annotations = [af.replace('.wav', '.onsets') for af in audio]
    
    data = []
    for aud, ann in zip(audio, annotations):
        if os.path.exists(aud) and os.path.exists(ann):
            data.append((aud, ann))
    
    return pd.DataFrame(data=data, columns=['audio', 'annotation'])

In [7]:
def make_output_path(base, outpath):
    
    root = os.path.splitext(base)[0]
    
    output = os.path.join(outpath, os.path.extsep.join([root, 'json']))
    
    return output

In [8]:
def analyze(dframe, outpath='/home/bmcfee/git/librosa_parameters/data/onset'):
    
    index = dframe.index[0]
    base = os.path.basename(dframe['audio'][index])
    
    outfile = make_output_path(base, outpath)
    
    if os.path.exists(outfile):
        print 'Cached {}'.format(base)
        data = pd.read_json(outfile, orient='records')
        return data
    else:
        print 'Processing {}'.format(base)
    
    # Load the truth
    ref_times = pd.read_table(dframe['annotation'][index], header=None, sep='\s+')[0].values

    # Load the audio
    y, sr = librosa.load(dframe['audio'][index])
    
    # Construct the output container
    results = []
    hop_length = 512
    
    effective_sr = sr // hop_length
    
    # Onset strength parameters
    for fmax in [8000, None]:
        for n_mels in [32, 64, 128]:
            S = librosa.feature.melspectrogram(y=y, sr=sr, fmax=fmax, n_mels=n_mels)
            
            for aggregate in [np.mean, np.median]:
        
                # Compute the onset detection function
                oenv = librosa.onset.onset_strength(S=S, sr=sr,
                                                    aggregate=aggregate)
                
                for delta in [0.0, 0.01, 0.02, 0.03, 0.05, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]:
                    onsets = librosa.onset.onset_detect(onset_envelope=oenv, sr=sr, delta=delta)
                    
                    params = {'aggregate': aggregate.__name__,
                              'fmax': fmax,
                              'n_mels': n_mels,
                              'delta': delta,}
                             
                    est_times = librosa.frames_to_time(onsets, sr=sr)
                    scores = mir_eval.onset.evaluate(ref_times, est_times)
                            
                    cont = OrderedDict(index=index)
                    cont.update(params)
                    cont.update(scores)
                    results.append(cont)
                    
    # Blow away the cache
    #librosa.cache.clear()
    data = pd.DataFrame.from_dict(results, orient='columns')
    data.to_json(outfile, orient='records')
        
    return data

In [9]:
def analyze_corpus(corpus):
    
    results = None
    for idx in corpus.index:
        new_results = analyze(corpus.loc[[idx]])
        if results is None:
            results = new_results
        else:
            results = pd.concat([results, new_results])
            
    return results

In [10]:
from joblib import Parallel, delayed

In [11]:
def p_analyze_corpus(corpus, n_jobs=3):
    
    results = None
    
    dfunc = delayed(analyze)
    
    results = Parallel(n_jobs=n_jobs, verbose=10)(dfunc(corpus.loc[[idx]])
                                                  for idx in corpus.index)

    return pd.concat(results)

---

In [12]:
onset_data = make_onset_corpus('/home/bmcfee/data/onsets/clean_data/')

In [13]:
onset_results = p_analyze_corpus(onset_data)

[Parallel(n_jobs=3)]: Done   1 out of  22 | elapsed:    0.0s remaining:    0.3s
[Parallel(n_jobs=3)]: Done   7 out of  37 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=3)]: Done   8 out of  38 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=3)]: Done  13 out of  41 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=3)]: Done  21 out of  49 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=3)]: Done  27 out of  67 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=3)]: Done  28 out of  68 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=3)]: Done  45 out of 115 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=3)]: Done  57 out of 151 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=3)]: Done  76 out of 188 | elapsed:    0.2s remaining:    0.4s
[Parallel(n_jobs=3)]: Done  77 out of 189 | elapsed:    0.2s remaining:    0.4s
[Parallel(n_jobs=3)]: Done  79 out of 197 | elapsed:    0.3s remaining:    0.4s
[Parallel(n_jobs=3)]: Done 101 out of 35

Cached ah_cello_03-Cello_Sonata_3__I_Allegro_ma_non_tanto_pt1.wav
Cached ah_cello_03_CelloTaksimi_pt1.wav
Cached ah_cello_08_-_Bach_(JS)-_Cello_Suite_-4_In_E_Flat_BWV_1010i_-_1._Preludium.wav
Cached ah_cello_14_VioloncelloTaksim_pt1.wav
Cached ah_clarinet_44784_alikirodgers_AB_Clarinet_01_pt1.wav
Cached ah_cello_cello1.wav
Cached ah_clarinet_44361_debudding_Clarinet_ORTF_Stereo_Pair_NT_5_s_01.wav
Cached ah_clarinet_clarinet1.wav
Cached ah_clarinet_my_clarinet1.wav
Cached ah_clarinet_SL1_pt1.wav
Cached ah_guitar_Guitar_Licks_07-06.wav
Cached ah_guitar_Guitar_Licks_06-12.wav
Cached ah_guitar_2684_TexasMusicForge_Dandelion_pt1.wav
Cached ah_guitar_Guitar_Licks_15-06.wav
Cached ah_guitar_Guitar_Licks_07-11.wav
Cached ah_guitar_Guitar_Licks_06-11.wav
Cached ah_guitar_guitar3.wav
Cached ah_guitar_Guitar_Licks_51-10.wav
Cached ah_guitar_Guitar_Licks_15-05.wav
Cached ah_guitar_my_guitar1.wav
Cached ah_guitar_guitar2.wav
Cached ah_guitar_Summer_Together_110_pt1.wav
Cached ah_kemence_10_huseyni_

In [16]:
onset_results.to_json('/home/bmcfee/git/librosa_parameters/onset_results.json', orient='records')

In [17]:
onset_results = pd.read_json('/home/bmcfee/git/librosa_parameters/onset_results.json', orient='records')

In [19]:
onset_scores = onset_results.groupby(['aggregate', 'fmax', 'n_mels', 'delta']).mean()

In [22]:
onset_scores['F-measure'].argmax()

(u'median', 8000.0, 128, 0.01)

In [24]:
best_f = onset_scores['F-measure'].argmax()

In [25]:
onset_scores.loc[best_f]

F-measure      0.313
Precision      0.360
Recall         0.402
index        175.000
Name: (median, 8000.0, 128, 0.01), dtype: float64

In [28]:
onset_scores

F-measure  Precision  Recall  index
aggregate fmax n_mels delta                                     
mean      8000 32     0.00       0.180      0.149   0.387    175
                      0.01       0.213      0.208   0.321    175
                      0.02       0.204      0.212   0.289    175
                      0.03       0.194      0.210   0.261    175
                      0.05       0.176      0.204   0.218    175
                      0.06       0.169      0.199   0.204    175
                      0.07       0.163      0.198   0.194    175
                      0.08       0.154      0.193   0.183    175
                      0.09       0.149      0.193   0.172    175
                      0.10       0.144      0.191   0.166    175
               64     0.00       0.180      0.149   0.381    175
                      0.01       0.215      0.209   0.323    175
                      0.02       0.205      0.213   0.289    175
                      0.03       0.195      0.211   0.265    175
                      0.05       0.178      0.206   0.221    175
                      0.06       0.169      0.200   0.205    175
                      0.07       0.163      0.199   0.191    175
                      0.08       0.155      0.194   0.181    175
                      0.09       0.151      0.195   0.172    175
                      0.10       0.145      0.192   0.164    175
               128    0.00       0.182      0.150   0.384    175
                      0.01       0.217      0.210   0.320    175
                      0.02       0.209      0.215   0.294    175
                      0.03       0.198      0.213   0.267    175
                      0.05       0.179      0.206   0.221    175
                      0.06       0.170      0.200   0.208    175
                      0.07       0.164      0.198   0.193    175
                      0.08       0.155      0.195   0.179    175
                      0.09       0.151      0.194   0.172    175
                      0.10       0.144      0.191   0.163    175
median    8000 32     0.00       0.234      0.213   0.434    175
                      0.01       0.258      0.295   0.352    175
                      0.02       0.244      0.294   0.316    175
                      0.03       0.229      0.288   0.287    175
                      0.05       0.210      0.288   0.247    175
                      0.06       0.201      0.286   0.237    175
                      0.07       0.192      0.280   0.225    175
                      0.08       0.185      0.281   0.216    175
                      0.09       0.178      0.278   0.204    175
                      0.10       0.173      0.279   0.192    175
               64     0.00       0.256      0.233   0.469    175
                      0.01       0.287      0.331   0.375    175
                      0.02       0.273      0.334   0.342    175
                      0.03       0.258      0.333   0.316    175
                      0.05       0.231      0.323   0.271    175
                      0.06       0.225      0.328   0.258    175
                      0.07       0.215      0.325   0.247    175
                      0.08       0.207      0.323   0.235    175
                      0.09       0.199      0.322   0.227    175
                      0.10       0.192      0.317   0.217    175
               128    0.00       0.271      0.244   0.498    175
                      0.01       0.313      0.360   0.402    175
                      0.02       0.299      0.366   0.369    175
                      0.03       0.287      0.369   0.344    175
                      0.05       0.265      0.370   0.306    175
                      0.06       0.256      0.371   0.294    175
                      0.07       0.245      0.367   0.282    175
                      0.08       0.229      0.360   0.260    175
                      0.09       0.220      0.359   0.247    175
                      0.10       0.211      0.356   0.234    175

In [27]:
# Previous configuration
onset_scores.loc[(u'mean', 8000, 128, 0.06)]

F-measure      0.170
Precision      0.200
Recall         0.208
index        175.000
Name: (mean, 8000.0, 128, 0.06), dtype: float64